# Multimodal 정의
- AI에서 modality는 "데이터형식"을 의미
- 멀티모달 AI는 여러 개의 데이터 형식을 가지고 수행하는 AI
    - input 양식 : 이미지 + 텍스트, 이미지 + 정형

## Mutlmodal 예시
### 음성인식
- 음성으로 입력된 정보를 인식하여 텍스트로 변환하는 기술
### 영상인식
- 영상에서 물체나 사람을 인식하고, 그에 대한 정보를 제공하는 기술
    - Google Lens : 카메라로 물체나 장면을 촬영하면 해당 물체나 자면에 대한 정보를 제공하는 서비스
    - Face ID : 얼굴 인식 기술을 사용하여 iPhon 잠금을 해제하는 기술
    - Selfie stick : 스마트폰을 부착하여 손을 자유롭게 사용할 수 있도록 만든 장치
### 자연어 처리
- 텍스트를 분석하여 의미를 파악하고, 그에 대한 응답을 생성하는 기술
    - 구글 번역 : 텍스트나 음성을 번역하는 서비스
    - 챗봇 : 사람과 대화하는 것처럼 사용자의 질문이나 요청에 응답하는 서비스
    - AI 작가 : 텍스트를 생성하는     
### 감정 인식
- 사람의 얼굴이나 표정이나 목소리에 감정을 인식하는 기술
    - 페이스북 감정 인식 : 사용자의 얼굴 표정에서 감정을 인식하여, 그에 맞는 광고를 노출하는 기술
    - 감정 인식 AI : 사용자의 감정을 인식하여, 그에 맞는 대응을 하는 AI
    - 감정 인식 로봇 : 사용자의 감정을 인시갛여, 그에 맞는 반응을 하는 로봇
### AR/VR
- 현실 세계에 가상의 정보를 결합하여 새로운 경험을 제공하는 기술
    - Pocketmon GO : AR 이용
    - 마인크래프트 : VR 이용
    - 메타버스 : 현실 세계와 가상 세계를 결한한 새로운 가상 세계


## Multimodal 종류
- Early Fusion : 종류가 다른 두가지 데이터를 하나의 데이터로 먼저 합친 이후 모델을 학습
- Late fusion : 종류가 다른 두가지 데이터를 각각 다른 모델에 학습 시킨 이후 나온 결과를 유향 (기존 앙상블 모델과 비슷한 방식)
- Joint or Intermediate Fusion : 두개의 모달리티 데이터를 동시에 학습하지 않고, 내가 원하는 모델의 깊이에서 모달리티를 병합할 수 있는 유연성 포함


## 사례 LLaVA
- 오픈소스 챗봇
- 이미지 해석과 대화 능력 금증
- 시각적 입력을 소화하고 지시 능력을 확인하기 위한 챗봇

# LLavA 실습 - 실패함.
- 코랩에서도 RAM 소진으로 실행 못해봄.

In [ ]:
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0

In [ ]:
import requests
from PIL import Image

image_url = "https://llava-vl.github.io/static/images/view.jpg"
image = Image.open(requests.get(image_url, stream=True).raw)
image

In [ ]:
!pip install torch torchvision
!pip uninstall timm -y
!pip install timm==0.9.12

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# 4bit qauntization
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
# Load Model using pipeline
from transformers import pipeline

model_id = "llava-hf/llava-1.5-7b-hf"

pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

# 양자화 안한 버전
# from transformers import pipeline

# pipe = pipeline("image-to-text", model="llava-hf/llava-1.5-7b-hf")

In [ ]:
max_new_tokens = 200
prompt = "USER: <image>\nWhat are the things I should be cautious about when I visit this place?\nASSISTANT:"

outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})

In [ ]:
print(outputs[0]["generated_text"])

# LlaVa 실습 2 - 직접 설치로 하는 방법 - 실패


In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/LLaVA
%cd /content/LLaVA

!pip install -q transformers==4.36.2
!pip install ninja
!pip install flash-attn --no-build-isolation

!pip install -e .

# !python -m llava.serve.cli \
#     --model-path liuhaotian/llava-v1.5-7b \
#     --image-file "https://llava-vl.github.io/static/images/view.jpg"

In [ ]:
# 추가 모드
# 모델을 다운로드 받아서 사용
!pip install huggingface_hub protobuf
%cd /content
from huggingface_hub import snapshot_download
snapshot_download(repo_id="liuhaotian/llava-v1.5-7b", local_dir="llava-v1.5-7b")

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import threading
import subprocess
threading.Thread(target=lambda: subprocess.run(['python3', '-m', 'llava.serve.controller', '--host', '0.0.0.0', '--port', '10000'], check=True), daemon=True).start()

In [ ]:
import threading
import subprocess

# Define the command to run the model worker
command = [
    'python3', '-m', 'llava.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40000',
    '--worker', 'http://localhost:40000',
    '--model-path', '/content/llava-v1.5-7b',
    '--load-8bit'
]

# Run the command in a separate thread
thread = threading.Thread(
    target=lambda: subprocess.run(command, check=True, shell=False),
    daemon=True
)
thread.start()

In [ ]:
!python3 -m llava.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share